# Introduction to the Keras Tuner

## 1. Overview

The Keras Tuner is a library that helps you pick the **optimal set of hyperparameters for your TensorFlow program**. The process of selecting the right set of hyperparameters for your machine learning (ML) application is called **hyperparameter tuning or hypertuning.**

Hyperparameters are the variables that govern the training process and the topology of an ML model. These variables remain constant over the training process and directly impact the performance of your ML program. Hyperparameters are of two types:

- **Model hyperparameters** which influence model selection such as the number and width of hidden layers
- **Algorithm hyperparameters** which influence the speed and quality of the learning algorithm such as the learning rate for Stochastic Gradient Descent (SGD) and the number of nearest neighbors for a k Nearest Neighbors (KNN) classifier

In this tutorial, you will use the Keras Tuner to perform hypertuning for an image classification application.

## 2. Setup

In [2]:
import tensorflow as tf
from tensorflow import keras
# pip install -q -U keras-tuner
import kerastuner as kt

print("The version of Tensorflow: {}".format(tf.__version__))
print("The version of KerasTuner: {}".format(kt.__version__))

The version of Tensorflow: 2.4.1
The version of KerasTuner: 1.0.2


## 3. Download and prepare the dataset

In [3]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [4]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

## 4. Define the model

In [5]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

## 5. Instantiate the tuner and perform hypertuning

In [6]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [7]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [8]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 29s]
val_accuracy: 0.8819166421890259

Best val_accuracy So Far: 0.8831666707992554
Total elapsed time: 00h 05m 49s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 192 and the optimal learning rate for the optimizer
is 0.001.



## 6. Train the model

In [9]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.6430 - accuracy: 0.7726 - val_loss: 0.4450 - val_accuracy: 0.8441
Epoch 2/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3922 - accuracy: 0.8570 - val_loss: 0.3723 - val_accuracy: 0.8698
Epoch 3/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3402 - accuracy: 0.8765 - val_loss: 0.3608 - val_accuracy: 0.8687
Epoch 4/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3117 - accuracy: 0.8836 - val_loss: 0.3503 - val_accuracy: 0.8743
Epoch 5/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2937 - accuracy: 0.8908 - val_loss: 0.3283 - val_accuracy: 0.8807
Epoch 6/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2805 - accuracy: 0.8955 - val_loss: 0.3215 - val_accuracy: 0.8853
Epoch 7/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2675 - accuracy: 0.8982 - val_loss: 0.3219 - val_accuracy:

In [10]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/26
1500/1500 [==============================] - 3s 2ms/step - loss: 0.6460 - accuracy: 0.7749 - val_loss: 0.4482 - val_accuracy: 0.8329
Epoch 2/26
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3943 - accuracy: 0.8599 - val_loss: 0.3613 - val_accuracy: 0.8712
Epoch 3/26
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3375 - accuracy: 0.8763 - val_loss: 0.3789 - val_accuracy: 0.8608
Epoch 4/26
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3134 - accuracy: 0.8846 - val_loss: 0.3512 - val_accuracy: 0.8714
Epoch 5/26
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2972 - accuracy: 0.8899 - val_loss: 0.3247 - val_accuracy: 0.8839
Epoch 6/26
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2697 - accuracy: 0.8991 - val_loss: 0.3211 - val_accuracy: 0.8844
Epoch 7/26
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2677 - accuracy: 0.8993 - val_loss: 0.3120 - val_accuracy:

In [11]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 2ms/step - loss: 0.3983 - accuracy: 0.8890
[test loss, test accuracy]: [0.3982964754104614, 0.8889999985694885]


## 7. Summary

### https://keras-team.github.io/keras-tuner/

### https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html

### https://keras.io/examples/

# reference

### https://tensorflow.google.cn/tutorials/keras/keras_tuner